# SPYNX - III phase PMSM motor control

<div class="alert bg-primary">This notebook will show control of a 3-phase AC motor using the EDPS (The Electric Drive Power Stage (EDPS) Board, a Trenz Electronic TEC0053, which is connected to the PYNQ-Z1 controller board for the evaluation..</div>

![](./images/motor.png)

 ## Objectives
 
 * [Access to Motor Control Parameters](#Step-2:-Instantiate-the-motor-control-object )
 * [Request Status Information of the Motor](#Step-4:-Read-status-registers)
 * [Programmatic Control of Motor](#Programmatic-Control-of-Motor)
 * [Continuous Status Capture from the Motor](#Step-5:-Allocate-DMA-for-stream-capture)
 * [Plots to Visualize Data Captured](#Plotting)
 * [Storing Captured Data for Analytics](#Dataframes-for-analytics)
 * [Live Interactive Plots to Investigate Data](#DASH-Demo)
 

### Step 1: Download the modified `EDDP` bitstream

In [ ]:
from pynq import Overlay
from pynq import MMIO
import numpy as np
overlay = Overlay("/usr/local/lib/python3.6/dist-packages/spynx/overlays/SPYNX.bit")
    #overlay.download()
    #overlay?
    #overlay._ip_map

### Step 2: Instantiate the motor control object 
#### Load the motor control library

In [ ]:
#import SPYNX library
from spynx.lib import *
motor = Motor_Controller()
print(f'Available motor modes : {motor.motor_modes}')
#motor?

In [ ]:
print(f'Memory mapped IO blocks : {motor.mmio_blocks}')

#### Initialization of the Encoder (find 0 position) - angle shift correction can be done here

In [ ]:
# initialize the encoder
import time
motor.set_mode('init_mode')
time.sleep(4)
# this offset needs to be tuned to fit the used motor
motor._write_controlreg(SHIFT.offset, 0)
motor.set_mode('reset_mode')

### Step 3: Set motor control mode and control using sliders

In [ ]:
# control widgets
from ipywidgets import interact, interactive, HBox, VBox, HTML
import ipywidgets as widgets

toggle = widgets.ToggleButton(description='Motor',button_style='success', layout = {'left' : '75px'})

mode = widgets.Dropdown(options=['Speed', 'Current'],  layout = {'left' : '50px','min_height':'45px'})

def clicked(toggle_0=toggle, mode=mode, RPM=None, Iq=None, lamda=None):
    if toggle_0:
        if clicked.prev_mode != mode:
            if mode == 'Speed':
                motor.set_mode('rpm_mode')
            else:
                motor.set_mode('torque_mode')
            clicked.prev_mode = mode # store mode
        motor.set_rpm(RPM)
        motor.set_torque(int(Iq/2.814433+.5))
        motor._write_controlreg(CONTROL_MPC.offset, int(lamda*126.2466+.5))
    else:
        motor.set_mode('reset_mode')
        clicked.prev_mode = -1
clicked.prev_mode = -1 # initialization

w = interactive(clicked,
                RPM = widgets.IntSlider(min=-4000, max=4000, step=1, value=1000),
                Iq = widgets.IntSlider(min=-1250, max=1250, step=1, value=300, description = 'Iq in mA'),
                lamda = widgets.FloatSlider(min=0, max=120, step=.79, value=0, description = '\u03BB_u *1000'))
labels = [widgets.Label('(only for FOC)'),
          widgets.Label('(Iq set point for FOC and MPC)'),
          widgets.Label('(switch cost wheigt for MPC)')]

caption = widgets.HTML("<b>Use the hardware switch SW0 to change between FOC and MPC.</b>",
                       layout = {'left' : '30px', 'max_width' : '400px'})
VBox([caption,
      HBox( w.children[:2]), 
      HBox([w.children[2], labels[0]]),
      HBox([w.children[3], labels[1]]),
      HBox([w.children[4], labels[2]])])

In [ ]:
motor._write_controlreg(TORQUE_SP.offset,0) #

In [ ]:
motor.set_mode('rpm_mode')
motor.set_rpm(2000)

In [ ]:
motor.set_mode('reset_mode')

In [ ]:
motor._read_controlreg(RPM_SP.offset)

In [ ]:
motor._read_controlreg(TORQUE_SP.offset)

[Back to Objectives](#Objectives)

### Step 4: Read status registers

In [ ]:
#read status registers 
motor_status = [(motor._read_controlreg(i + ANGLE.offset)) for i in
                range(0, 16, 4)]
high_sp, low_sp = bytesplit(motor_status[1])
high_id, low_id = bytesplit(motor_status[2])
high_iq, low_iq = bytesplit(motor_status[3])
print(f'Angle in degrees : {motor_status[0] * 0.36}')
print(f'Angle in steps per thousand: {(motor_status[0])}')
print(f'Id : {np.int16(low_id) * 2.814433} mAmp')
print(f'Iq : {np.int16(low_iq) * 2.814433} mAmp')
print(f'Speed in RPM : {-(np.int16(low_sp))}')

[Back to Objectives](#Objectives)

### Programmatic Control of Motor

In [ ]:
import time
for i in range(5, -6, -1):
        motor.set_mode('rpm_mode')
        motor.set_rpm(i*1000)
        print("n_target = " + str(i*1000) + "rpm")
        time.sleep(.5)
        motor_status = [(motor._read_controlreg(SPEED.offset))]
        high_sp, low_sp = bytesplit(motor_status[0])
        print(f'real RPM : {-(np.int16(low_sp))}')
        time.sleep(.5)
motor.stop()

[Back to Objectives](#Objectives)

### Step 5: Allocate DMA for stream capture

In [ ]:
# Allocate DMA Memory
from pynq import Xlnk

xlnk = Xlnk()
input_buffer = xlnk.cma_array(shape=(256,), dtype=np.uint8)

capture_address = input_buffer.physical_address
#print(f'Physical Address of data stream capture: {hex(capture_address)}')

### Step 6: Log stream data as per control mode

In [ ]:
# Capture Interface Widgets
import time
from pynq import MMIO
from ipywidgets import interact, interactive, HBox, VBox, HTML
import ipywidgets as widgets

mode = widgets.ToggleButtons(
    options=[ 'Ia / Ib (filtered)', 'Ia / Ib (raw)',  
              'Ialpha / Ibeta', 'Id / Iq','Vd / Vq', 
              'Valpha / Vbeta', 'Va / Vb / Vc', 'V_PWM', ],
    description='Capture:',
    button_style= '',
    tooltips=['Capture Ia, Ib, RPM and Angle (filtered)', 
              'Capture Ia, Ib, RPM and Angle (raw)', 
              'Capture Ialpha, Ibeta, RPM and Angle', 
              'Capture Id, Iq, RPM and Angle', 
              'Capture Vd, Vq, RPM and Angle', 
              'Capture Valpha, Vbeta, RPM and Angle', 
              'Capture Va, Vb and Vc', 
              'Capture Va, Vb and Vc with SVPWM'],
    layout={'width': '700px'})

# e.g. decimation 2 means only every 2nd sample is captured, 3 -> only every 3rd
decimation = widgets.IntSlider(
    value=2,
    min=1,
    max=100,
    description='Decimation:',
    readout_format='d',
    layout = {'height': '50px', 'width' : '300px'})

sample_count = widgets.IntSlider(
    value=4096,
    min=32,
    max=8192,
    step = 32,
    description='Sample Count:',
    readout_format='d',
    layout = {'height': '50px', 'width' : '400px'},
    style = {'description_width' : '100px'})

Label0 = widgets.Label(
    value = "Ready for next Capture.",
    layout={'width': '300px'})

cap_button = widgets.Button(
    description='Start the Capture',
    button_style='warning',
    tooltip='Capture')

cap_progress = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    bar_style='info',
    layout={'width': '700px'}  )

load_button = widgets.Button(
    description='Load the Capture',
    button_style='',
    disabled = True,
    tooltip='Read the captured data from the FIFO',
    layout={'left': '300px'})

display(VBox([mode,
              HBox([decimation, sample_count]),
              VBox([HBox([Label0, cap_button]), cap_progress]),
              load_button]))


def fill_FIFO(b):
    b.disabled = True
    motor._write_controlreg(CONTROL_REG2.offset, mode.index)
    motor._write_controlreg(DECIMATION.offset, decimation.value-1)
    global capture_count 
    capture_count = int(sample_count.value/32)
    capture_time = capture_count*(decimation.value)/625000
    Label0.value = "Capturing - wait for " + str(capture_time) + " seconds..."
    # set ready Signal of Capture unit to low -> FIFO stores all data until it's full
    motor.write_capturereg(0,2)
    t_start = time.time()
    while cap_progress.value < cap_progress.max:
        cap_progress.value = int(100*(time.time()-t_start)/capture_time)
    Label0.value = "Capture is stored in FIFO and ready to be read."
    cap_progress.bar_style = 'success'
    load_button.disabled = False

def load_data(b):
    b.disabled = True
    global Signal_1, Signal_2, Signal_3, Label_1, Label_2, Label_3, angle, rpm, YLabel
    for name in list(globals().keys()):
        if name.startswith('Signal_') or name.startswith('Label_') or name == 'angle' or name == 'rpm':
            del globals()[name]
    def continuous_capture(capture_count):    
        mmio_stream = MMIO(capture_address, 256)
        cap_list = [([]) for i in range(4)]
        for _ in range(capture_count):
            motor.stream_capture(capture_address)
            for i in range(4, 260, 4):
                stream = mmio_stream.read(i - 4, 4)
                highbits, lowbits = bytesplit(stream)
                if (i % 8 != 0):
                    cap_list[0].extend([(np.int16(lowbits))])
                    cap_list[1].extend([(np.int16(highbits))])
                else:
                    cap_list[2].extend([(np.int16(lowbits))])
                    cap_list[3].extend([(np.int16(highbits))])
        return cap_list

    cap_list = continuous_capture(capture_count)
    #Sig1, Sig2, rpm, angle  = cap_list[0], cap_list[1], cap_list[2], cap_list[3]

    cap_mode = motor._read_controlreg(CONTROL_REG2.offset)
    if cap_mode <= 3:
        if cap_mode == 1:
            # for the raw adc current values an offset has to be subtracted
            Signal_1 = (np.array(cap_list[0])-247) * 2.814433
            Signal_2 = (np.array(cap_list[1])-130) * 2.814433
            Signal_3 = -Signal_1 - Signal_2
            Label_1 = 'I_a_raw'
            Label_2 = 'I_b_raw'
            Label_3 = 'I_c_raw'
        else:
            Signal_1 = np.array(cap_list[0]) * 2.814433
            Signal_2 = np.array(cap_list[1]) * 2.814433
            if cap_mode == 2:
                Label_1 = 'I_alpha'
                Label_2 = 'I_beta'
            elif cap_mode == 3:
                Label_1 = 'I_d'
                Label_2 = 'I_q'
            elif cap_mode == 0:
                Signal_3 = -Signal_1 - Signal_2
                Label_1 = 'I_a_filtered'
                Label_2 = 'I_b_filtered'
                Label_3 = 'I_c_filtered'
        rpm, angle = cap_list[2], cap_list[3]
        YLabel = 'Current in mA'
    else:
        if cap_mode == 4:
            Label_1 = 'V_d'
            Label_2 = 'V_q'
            angle =  cap_list[2]
        elif cap_mode == 5:
            Label_1 = 'V_alpha'
            Label_2 = 'V_beta'
            angle =  cap_list[2]
        elif cap_mode == 6:
            Label_1 = 'V_a'
            Label_2 = 'V_b'
            Label_3 = 'V_c'
            angle =  cap_list[3]
            Signal_3 = np.array(cap_list[2])*0.0002113

        elif cap_mode == 7:
            Label_1 = 'V_a_PWM'
            Label_2 = 'V_b_PWM'
            Label_3 = 'V_c_PWM'
            Signal_3 = np.array(cap_list[2])*0.0002113
        Signal_1 = np.array(cap_list[0]) * 0.0002113
        Signal_2 = np.array(cap_list[1]) * 0.0002113
        YLabel = 'Voltage in V'
    motor.write_capturereg(0,0) # reset the Capture-IP
    cap_progress.value, cap_progress.bar_style = 0, 'info'
    Label0.value = "Ready for next Capture."
    cap_button.disabled = False

cap_button.on_click(fill_FIFO)
load_button.on_click(load_data)

[Back to Objectives](#Objectives)

### Step 7: Plotting

#### Load Plot Library

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

#### Signal_1 and Signal_2 vs Sample count

In [ ]:
# plot code
fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(111)
ax.plot(Signal_1)
ax.plot(Signal_2)
#ax.plot(Signal_1-Signal_2)
plt.legend((Label_1, Label_2), loc='upper right')
ax.set_ylabel(YLabel)
plt.show()

#### Signal_1 vs Signal_2

In [ ]:
# plot code
fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(111)
ax.scatter(Signal_1, Signal_2)
#ax.scatter(Signal_1, Signal_3)
#ax.scatter(Signal_2, Signal_3)
plt.ylabel(Label_2)
plt.xlabel(Label_1)
plt.show()

#### Signal_1, Signal_2 and Signal_3 vs angle

In [ ]:
# plot code
import math
fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(111)
ax.scatter(angle, Signal_1)
ax.scatter(angle, Signal_2)
ax.scatter(angle, Signal_3)
sin_SP = np.empty((1000))
tq_sp = 312#motor._read_controlreg(TORQUE_SP.offset)
for idx in range(1000):
    sin_SP[idx] = -math.sin(idx/250*math.pi)*tq_sp*2.814
ax.plot(sin_SP)
ax.plot()
plt.legend((Label_1, Label_2, Label_3), loc='upper right')
ax.set_ylabel(YLabel)
plt.xlabel('angle in 4*pi/999')
plt.show()

#### RPM and Angle vs Sample count

In [ ]:
# plot code
fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(111)
ax.plot(angle)
ax.plot(np.array(rpm))
plt.legend(('angle', 'rpm'), loc='upper right')
plt.show()

[Back to Objectives](#Objectives)

### Step 8: Stop motor and reset xlnk

In [ ]:
xlnk.xlnk_reset()
motor.stop()

### Dataframes for analytics

In [ ]:
import pandas as pd

data = {'Ia' : Signal_1,
        'Ib' : Signal_2,
        'angle': angle,
        'rpm':  rpm}

df = pd.DataFrame(data, columns = ['Ia', 'Ib', 'angle', 'rpm'])
df

In [ ]:
df.to_csv('motor_data.txt')

[Back to Objectives](#Objectives)